In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [2]:
!kaggle competitions download -c covid-19-mrna-vaccine-degradation-41

 84% 7.00M/8.29M [00:00<00:00, 13.5MB/s]
100% 8.29M/8.29M [00:00<00:00, 9.87MB/s]


In [3]:
!unzip covid-19-mrna-vaccine-degradation-41.zip

Archive:  covid-19-mrna-vaccine-degradation-41.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


***
# library

In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
import matplotlib.pylab as plt
plt.style.use("seaborn")

<ipython-input-4-501f54de0c93>:11: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use("seaborn")


In [26]:
def mcrmse_loss(y_true, y_pred, N=3):

    assert len(y_true) == len(y_pred)
    n = len(y_true)
    return np.sum(np.sqrt(np.sum((y_true - y_pred)**2, axis=0)/n)) / N

***
### Read the data


In [28]:
train = pd.read_csv('train.csv')
train.T

,0,1,2,3,4,5,6,7,8,9,...,130550,130551,130552,130553,130554,130555,130556,130557,130558,130559
id,id_159f0ff46,id_d73546f4e,id_f3037fedf,id_468300749,id_69e3878a2,id_71309e53b,id_d242af194,id_6b0b9ac67,id_32c5d2db6,id_5ab699511,...,id_485abbecd,id_d380bb370,id_db19924fa,id_dd32a1449,id_d8855591b,id_796128149,id_8a466e770,id_998669e76,id_6fe4a1abc,id_5b9c7041f
id_seqpos,id_159f0ff46_44,id_d73546f4e_38,id_f3037fedf_34,id_468300749_12,id_69e3878a2_28,id_71309e53b_58,id_d242af194_27,id_6b0b9ac67_11,id_32c5d2db6_55,id_5ab699511_6,...,id_485abbecd_16,id_d380bb370_2,id_db19924fa_35,id_dd32a1449_57,id_d8855591b_17,id_796128149_27,id_8a466e770_39,id_998669e76_52,id_6fe4a1abc_45,id_5b9c7041f_51
sequence,A,G,G,G,U,G,A,C,C,U,...,U,A,C,A,G,U,A,A,G,C
structure,.,.,(,(,(,.,.,(,),(,...,(,.,(,.,.,),.,.,.,.
predicted_loop_type,H,I,S,S,S,H,B,S,S,S,...,S,E,S,H,I,S,X,H,I,I
reactivity,1.7786,0.4397,0.112,0.0622,0.1378,0.6894,3.8229,0.4133,0.0569,1.4606,...,0.0,0.7224,0.0,0.6758,0.5207,0.1244,0.0201,0.4597,0.3634,0.1458
reactivity_error,0.1195,0.1033,0.1375,0.0471,0.1352,0.0955,0.175,0.0482,0.0719,0.604,...,0.1621,0.3017,0.1115,0.088,0.1644,0.0393,0.0351,0.0601,0.1063,0.0427
deg_Mg_pH10,0.7346,0.163,0.0,0.1179,0.4069,0.7169,1.9249,0.2761,0.1999,2.2465,...,0.234,0.5424,0.0,0.3816,0.4049,0.2672,0.1271,0.2014,0.1928,0.4301
deg_error_Mg_pH10,0.079,0.0799,0.0744,0.1075,0.2168,0.1013,0.1502,0.0497,0.0877,0.7787,...,0.4712,0.3707,0.132,0.0627,0.1745,0.0728,0.0583,0.0515,0.0738,0.0687
deg_pH10,0.368,0.0878,0.1835,0.0657,0.3375,0.7182,0.7902,0.1711,0.0464,0.9501,...,0.0,0.4352,0.3399,0.3116,0.2789,0.1479,0.058,0.2838,0.1885,0.2811


In [29]:

test = pd.read_csv('test.csv')
test

,id,id_seqpos,sequence,structure,predicted_loop_type,reactivity_error,deg_error_Mg_pH10,deg_pH10,deg_error_pH10,deg_error_Mg_50C,...,b4_structure,a4_structure,b4_predicted_loop_type,a4_predicted_loop_type,b5_sequence,a5_sequence,b5_structure,a5_structure,b5_predicted_loop_type,a5_predicted_loop_type
0,id_001f94081,id_001f94081_0,G,.,E,0.1359,0.2613,2.3375,0.2631,0.1501,...,-1,.,-1,E,-1,A,-1,(,-1,S
1,id_001f94081,id_001f94081_4,A,.,E,0.1314,0.1798,0.2635,0.1000,0.1369,...,.,(,E,S,-1,C,-1,(,-1,S
2,id_001f94081,id_001f94081_8,U,(,S,0.0756,0.1056,0.0530,0.0517,0.0705,...,.,.,E,H,A,U,.,.,E,H
3,id_001f94081,id_001f94081_9,C,(,S,0.1087,0.1896,0.5348,0.1474,0.1588,...,(,.,S,H,A,A,.,.,E,H
4,id_001f94081,id_001f94081_13,U,.,H,0.1388,0.1810,0.7699,0.1417,0.1382,...,(,.,S,H,U,G,(,),S,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32635,id_fff546103,id_fff546103_39,A,(,S,0.0091,0.0344,0.0140,0.0303,0.0284,...,(,(,S,S,C,A,(,.,S,H
32636,id_fff546103,id_fff546103_52,A,),S,0.0237,0.0390,0.0806,0.0529,0.0361,...,),),S,S,G,G,),),S,S
32637,id_fff546103,id_fff546103_62,A,.,B,0.0655,0.0672,0.5538,0.1022,0.0694,...,.,),M,S,G,C,),),S,S
32638,id_fff546103,id_fff546103_65,A,),S,0.0218,0.0391,0.1642,0.0611,0.0520,...,),(,S,S,U,C,),(,S,S


***
## Data Preprocess

### more detail about the data


#### `Train`
1. numeric

2. categorical

In [ ]:
train.describe()

,reactivity,reactivity_error,deg_Mg_pH10,deg_error_Mg_pH10,deg_pH10,deg_error_pH10,deg_Mg_50C,deg_error_Mg_50C,deg_50C,deg_error_50C
count,130560.000000,130560.000000,130560.00000,130560.000000,130560.000000,130560.000000,130560.000000,130560.000000,130560.000000,130560.000000
mean,0.376570,2825.710861,0.44650,2332.806124,0.448538,6288.673592,0.405686,3579.976118,0.427428,5969.595514
std,0.730013,19105.915817,0.70582,14614.719469,1.300353,42323.269166,0.840844,23229.704416,1.126904,38182.352844
min,-20.664800,0.001900,-14.74050,0.006100,-44.515300,0.008300,-24.252500,0.004100,-38.339900,0.006500
25%,0.043800,0.044900,0.13130,0.057200,0.099900,0.073900,0.074100,0.052000,0.092700,0.081300
50%,0.216200,0.083000,0.28990,0.098800,0.276100,0.120500,0.256800,0.088800,0.308800,0.133700
75%,0.540400,0.172500,0.56760,0.200000,0.569100,0.244200,0.560000,0.181600,0.613825,0.265325
max,20.667500,146151.225000,14.74010,104235.174200,44.521200,314833.569500,24.252500,171525.321700,38.343800,271158.604000


In [ ]:
train.describe(include=[object]).T

,count,unique,top,freq
id,130560,2400,id_c03e77453,63
id_seqpos,130560,130560,id_159f0ff46_44,1
sequence,130560,4,A,47922
structure,130560,3,.,66648
predicted_loop_type,130560,7,S,63912
b1_sequence,130560,5,A,46512
a1_sequence,130560,4,A,47936
b1_structure,130560,4,.,65243
a1_structure,130560,3,.,64786
b1_predicted_loop_type,130560,8,S,63413


In [ ]:
train.isnull().sum()

id                        0
id_seqpos                 0
sequence                  0
structure                 0
predicted_loop_type       0
reactivity                0
reactivity_error          0
deg_Mg_pH10               0
deg_error_Mg_pH10         0
deg_pH10                  0
deg_error_pH10            0
deg_Mg_50C                0
deg_error_Mg_50C          0
deg_50C                   0
deg_error_50C             0
b1_sequence               0
a1_sequence               0
b1_structure              0
a1_structure              0
b1_predicted_loop_type    0
a1_predicted_loop_type    0
b2_sequence               0
a2_sequence               0
b2_structure              0
a2_structure              0
b2_predicted_loop_type    0
a2_predicted_loop_type    0
b3_sequence               0
a3_sequence               0
b3_structure              0
a3_structure              0
b3_predicted_loop_type    0
a3_predicted_loop_type    0
b4_sequence               0
a4_sequence               0
b4_structure        

### counts_vector

In [ ]:

# List of categorical columns
categorical_columns = ['sequence', 'structure', 'predicted_loop_type', 'b1_structure', 'a1_structure',
                       'b2_structure', 'a2_structure', 'b3_structure', 'a3_structure', 'b4_structure',
                       'a4_structure', 'b5_structure', 'a5_structure', 'b1_predicted_loop_type',
                       'a1_predicted_loop_type', 'b2_predicted_loop_type', 'a2_predicted_loop_type',
                       'b3_predicted_loop_type', 'a3_predicted_loop_type', 'b4_predicted_loop_type',
                       'a4_predicted_loop_type', 'b5_predicted_loop_type', 'a5_predicted_loop_type']


In [ ]:
numerical_columns = [col for col in train.columns if col not in categorical_columns]

In [ ]:
numerical_columns

['id',
 'id_seqpos',
 'reactivity',
 'reactivity_error',
 'deg_Mg_pH10',
 'deg_error_Mg_pH10',
 'deg_pH10',
 'deg_error_pH10',
 'deg_Mg_50C',
 'deg_error_Mg_50C',
 'deg_50C',
 'deg_error_50C',
 'b1_sequence',
 'a1_sequence',
 'b2_sequence',
 'a2_sequence',
 'b3_sequence',
 'a3_sequence',
 'b4_sequence',
 'a4_sequence',
 'b5_sequence',
 'a5_sequence']

In [ ]:
train['sequence'].value_counts()

A    47922
G    34975
C    24678
U    22985
Name: sequence, dtype: int64

In [ ]:
train['structure'].value_counts()

.    66648
(    31956
)    31956
Name: structure, dtype: int64

In [ ]:
train['predicted_loop_type'].value_counts()

S    63912
E    28225
H    15240
I     9729
X     6372
M     4651
B     2431
Name: predicted_loop_type, dtype: int64

In [ ]:
train['b1_structure'].value_counts()

.     65243
(     31948
)     31465
-1     1904
Name: b1_structure, dtype: int64

In [ ]:
train['a1_structure'].value_counts()

.    64786
(    33869
)    31905
Name: a1_structure, dtype: int64

In [ ]:
train['b2_structure'].value_counts()

.     64170
(     31928
)     30659
-1     3803
Name: b2_structure, dtype: int64

In [ ]:
train['a2_structure'].value_counts()

.    62832
(    35837
)    31891
Name: a2_structure, dtype: int64

In [ ]:
train['b3_structure'].value_counts()

.     63274
(     31861
)     29680
-1     5745
Name: b3_structure, dtype: int64

In [ ]:
train['a3_structure'].value_counts()

.    60889
(    37767
)    31904
Name: a3_structure, dtype: int64

In [ ]:
train['b4_structure'].value_counts()

.     62490
(     31909
)     28497
-1     7664
Name: b4_structure, dtype: int64

In [ ]:
train['a4_structure'].value_counts()

.    58929
(    39635
)    31996
Name: a4_structure, dtype: int64

In [ ]:
train['b5_structure'].value_counts()

.     61603
(     32009
)     27365
-1     9583
Name: b5_structure, dtype: int64

In [ ]:
train['a5_structure'].value_counts()

.    57026
(    41569
)    31965
Name: a5_structure, dtype: int64

In [ ]:
train['b1_predicted_loop_type'].value_counts()

S     63413
E     28019
H     15195
I      9760
X      5425
M      4398
B      2446
-1     1904
Name: b1_predicted_loop_type, dtype: int64

In [ ]:
train['a1_predicted_loop_type'].value_counts()

S    65774
E    26316
H    15262
I     9776
X     6373
M     4626
B     2433
Name: a1_predicted_loop_type, dtype: int64

In [ ]:
train['b2_predicted_loop_type'].value_counts()

S     62587
E     27845
H     15241
I      9703
X      4737
M      4200
-1     3803
B      2444
Name: b2_predicted_loop_type, dtype: int64

In [ ]:
train['a2_predicted_loop_type'].value_counts()

S    67728
E    24375
H    15242
I     9736
X     6375
M     4642
B     2462
Name: a2_predicted_loop_type, dtype: int64

In [ ]:
train['b3_predicted_loop_type'].value_counts()

S     61541
E     27591
H     15282
I      9670
-1     5745
X      4232
M      4047
B      2452
Name: b3_predicted_loop_type, dtype: int64

In [ ]:
train['a3_predicted_loop_type'].value_counts()

S    69671
E    22440
H    15267
I     9738
X     6360
M     4607
B     2477
Name: a3_predicted_loop_type, dtype: int64

In [ ]:
train['b4_predicted_loop_type'].value_counts()

S     60406
E     27364
H     15262
I      9601
-1     7664
M      3944
X      3857
B      2462
Name: b4_predicted_loop_type, dtype: int64

In [ ]:
train['a4_predicted_loop_type'].value_counts()

S    71631
E    20539
H    15207
I     9701
X     6413
M     4605
B     2464
Name: a4_predicted_loop_type, dtype: int64

In [ ]:
train['b5_predicted_loop_type'].value_counts()

S     59374
E     27132
H     15190
-1     9583
I      9451
M      3863
X      3585
B      2382
Name: b5_predicted_loop_type, dtype: int64

In [ ]:
train['a5_predicted_loop_type'].value_counts()

S    73534
E    18612
H    15199
I     9761
X     6394
M     4592
B     2468
Name: a5_predicted_loop_type, dtype: int64

In [ ]:
test['b5_predicted_loop_type'].value_counts()

S     14865
E      6710
H      3728
-1     2417
I      2389
M      1000
X       895
B       636
Name: b5_predicted_loop_type, dtype: int64

***
### encoded


In [18]:
# train_want = train.drop(['reactivity', 'deg_Mg_pH10', 'deg_Mg_50C'],axis=1)

In [19]:
# numeric_columns = train_want.select_dtypes(include=['int64', 'float64']).columns
# numeric_columns_test = test.select_dtypes(include=['int64', 'float64']).columns

In [21]:
# from sklearn.preprocessing import StandardScaler
# # Initialize the StandardScaler
# scaler = StandardScaler()
# # Fit the scaler on the training data and transform both training and testing data
# train[numeric_columns] = scaler.fit_transform(train[numeric_columns])
# test[numeric_columns] = scaler.transform(test[numeric_columns])


In [22]:
train

,id,id_seqpos,sequence,structure,predicted_loop_type,reactivity,reactivity_error,deg_Mg_pH10,deg_error_Mg_pH10,deg_pH10,...,b4_structure,a4_structure,b4_predicted_loop_type,a4_predicted_loop_type,b5_sequence,a5_sequence,b5_structure,a5_structure,b5_predicted_loop_type,a5_predicted_loop_type
0,id_159f0ff46,id_159f0ff46_44,A,.,H,1.7786,-0.147891,0.7346,-0.159616,-0.061936,...,(,),S,S,C,A,.,),I,S
1,id_d73546f4e,id_d73546f4e_38,G,.,I,0.4397,-0.147892,0.1630,-0.159615,-0.277417,...,(,.,S,H,C,C,.,.,B,H
2,id_f3037fedf,id_f3037fedf_34,G,(,S,0.1120,-0.147891,0.0000,-0.159616,-0.203821,...,(,(,S,S,A,G,(,(,S,S
3,id_468300749,id_468300749_12,G,(,S,0.0622,-0.147895,0.1179,-0.159614,-0.294412,...,.,(,E,S,A,U,.,(,E,S
4,id_69e3878a2,id_69e3878a2_28,U,(,S,0.1378,-0.147891,0.4069,-0.159606,-0.085391,...,(,(,S,S,G,G,(,.,S,H
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130555,id_796128149,id_796128149_27,U,),S,0.1244,-0.147896,0.2672,-0.159616,-0.231198,...,),),S,S,A,A,.,.,H,M
130556,id_8a466e770,id_8a466e770_39,A,.,X,0.0201,-0.147896,0.1271,-0.159617,-0.300334,...,.,.,H,X,A,U,.,.,H,X
130557,id_998669e76,id_998669e76_52,A,.,H,0.4597,-0.147895,0.2014,-0.159617,-0.126688,...,.,.,H,I,U,U,.,),H,S
130558,id_6fe4a1abc,id_6fe4a1abc_45,G,.,I,0.3634,-0.147892,0.1928,-0.159616,-0.199976,...,),.,S,I,G,G,),),S,S


In [30]:
le = LabelEncoder()

train[['sequence', 'structure', 'predicted_loop_type', 'b1_structure', 'a1_structure',
                       'b2_structure', 'a2_structure', 'b3_structure', 'a3_structure', 'b4_structure',
                       'a4_structure', 'b5_structure', 'a5_structure', 'b1_predicted_loop_type',
                       'a1_predicted_loop_type', 'b2_predicted_loop_type', 'a2_predicted_loop_type',
                       'b3_predicted_loop_type', 'a3_predicted_loop_type', 'b4_predicted_loop_type',
                       'a4_predicted_loop_type', 'b5_predicted_loop_type', 'a5_predicted_loop_type','b1_sequence', 'a1_sequence', 'b2_sequence', 'a2_sequence', 'b3_sequence', 'a3_sequence', 'b4_sequence', 'a4_sequence', 'b5_sequence', 'a5_sequence']]=train[['sequence', 'structure', 'predicted_loop_type', 'b1_structure', 'a1_structure',
                       'b2_structure', 'a2_structure', 'b3_structure', 'a3_structure', 'b4_structure',
                       'a4_structure', 'b5_structure', 'a5_structure', 'b1_predicted_loop_type',
                       'a1_predicted_loop_type', 'b2_predicted_loop_type', 'a2_predicted_loop_type',
                       'b3_predicted_loop_type', 'a3_predicted_loop_type', 'b4_predicted_loop_type',
                       'a4_predicted_loop_type', 'b5_predicted_loop_type', 'a5_predicted_loop_type','b1_sequence', 'a1_sequence', 'b2_sequence', 'a2_sequence', 'b3_sequence', 'a3_sequence', 'b4_sequence', 'a4_sequence', 'b5_sequence', 'a5_sequence']].apply(le.fit_transform)

In [31]:
le = LabelEncoder()

test[['sequence', 'structure', 'predicted_loop_type', 'b1_structure', 'a1_structure',
                       'b2_structure', 'a2_structure', 'b3_structure', 'a3_structure', 'b4_structure',
                       'a4_structure', 'b5_structure', 'a5_structure', 'b1_predicted_loop_type',
                       'a1_predicted_loop_type', 'b2_predicted_loop_type', 'a2_predicted_loop_type',
                       'b3_predicted_loop_type', 'a3_predicted_loop_type', 'b4_predicted_loop_type',
                       'a4_predicted_loop_type', 'b5_predicted_loop_type', 'a5_predicted_loop_type','b1_sequence', 'a1_sequence', 'b2_sequence', 'a2_sequence', 'b3_sequence', 'a3_sequence', 'b4_sequence', 'a4_sequence', 'b5_sequence', 'a5_sequence']]=test[['sequence', 'structure', 'predicted_loop_type', 'b1_structure', 'a1_structure',
                       'b2_structure', 'a2_structure', 'b3_structure', 'a3_structure', 'b4_structure',
                       'a4_structure', 'b5_structure', 'a5_structure', 'b1_predicted_loop_type',
                       'a1_predicted_loop_type', 'b2_predicted_loop_type', 'a2_predicted_loop_type',
                       'b3_predicted_loop_type', 'a3_predicted_loop_type', 'b4_predicted_loop_type',
                       'a4_predicted_loop_type', 'b5_predicted_loop_type', 'a5_predicted_loop_type','b1_sequence', 'a1_sequence', 'b2_sequence', 'a2_sequence', 'b3_sequence', 'a3_sequence', 'b4_sequence', 'a4_sequence', 'b5_sequence', 'a5_sequence']].apply(le.fit_transform)

In [17]:
train['b5_predicted_loop_type'].value_counts()

6    59374
2    27132
3    15190
0     9583
4     9451
5     3863
7     3585
1     2382
Name: b5_predicted_loop_type, dtype: int64

In [ ]:
test['b1_structure'].value_counts()

3    16269
0     8033
1     7842
2      496
Name: b1_structure, dtype: int64

In [ ]:
test['b2_structure'].value_counts()

3    15975
0     8053
1     7615
2      997
Name: b2_structure, dtype: int64

***
### model

In [35]:
# train = train.drop(['id', 'id_seqpos', 'reactivity_error', 'deg_error_Mg_pH10', 'deg_error_pH10', 'deg_error_Mg_50C', 'deg_error_50C',  'deg_pH10', 'deg_50C'], axis=1)

In [36]:
# test = test.drop([ 'reactivity_error', 'deg_error_Mg_pH10', 'deg_error_pH10', 'deg_error_Mg_50C', 'deg_error_50C',  'deg_pH10', 'deg_50C'], axis=1)

In [32]:
train = train.drop(['id', 'id_seqpos'],axis=1)

In [33]:
test = test.drop(['sequence','structure','predicted_loop_type'], axis=1)

In [34]:
train = train.drop(['sequence','structure','predicted_loop_type'], axis=1)

In [35]:
train

,reactivity,reactivity_error,deg_Mg_pH10,deg_error_Mg_pH10,deg_pH10,deg_error_pH10,deg_Mg_50C,deg_error_Mg_50C,deg_50C,deg_error_50C,...,b4_structure,a4_structure,b4_predicted_loop_type,a4_predicted_loop_type,b5_sequence,a5_sequence,b5_structure,a5_structure,b5_predicted_loop_type,a5_predicted_loop_type
0,1.7786,0.1195,0.7346,0.0790,0.3680,0.0851,0.7228,0.0792,0.5685,0.1051,...,0,1,6,5,2,0,3,1,4,5
1,0.4397,0.1033,0.1630,0.0799,0.0878,0.0878,0.1100,0.0683,0.0666,0.0864,...,0,2,6,2,2,1,3,2,1,2
2,0.1120,0.1375,0.0000,0.0744,0.1835,0.1969,0.0933,0.1163,0.0000,0.1088,...,0,0,6,5,1,2,0,0,6,5
3,0.0622,0.0471,0.1179,0.1075,0.0657,0.0615,0.0548,0.0506,0.0749,0.0913,...,3,0,2,5,1,3,3,0,2,5
4,0.1378,0.1352,0.4069,0.2168,0.3375,0.2623,0.2378,0.1569,0.6247,0.3370,...,0,0,6,5,3,2,0,2,6,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130555,0.1244,0.0393,0.2672,0.0728,0.1479,0.0493,0.1921,0.0481,0.4711,0.0883,...,1,1,6,5,1,0,3,2,3,4
130556,0.0201,0.0351,0.1271,0.0583,0.0580,0.0660,0.0245,0.0419,0.2559,0.1039,...,3,2,3,6,1,3,3,2,3,6
130557,0.4597,0.0601,0.2014,0.0515,0.2838,0.0793,0.2042,0.0528,0.3089,0.0861,...,3,2,3,3,4,3,3,1,3,5
130558,0.3634,0.1063,0.1928,0.0738,0.1885,0.1276,0.1627,0.0773,0.1001,0.1234,...,1,2,6,3,3,2,1,1,6,5


In [36]:
# Split data into features (X) and target variables (y)
X = train.drop(['reactivity', 'deg_Mg_pH10', 'deg_Mg_50C'], axis=1)
y = train[['reactivity', 'deg_Mg_pH10', 'deg_Mg_50C']]

# # Split data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [37]:
X.shape,y.shape

((130560, 37), (130560, 3))

In [38]:
# Define the number of samples for training
train_samples = 104448

# Split data into training and testing sets
X_train, X_test = X[:train_samples], X[train_samples:]
y_train, y_test = y[:train_samples], y[train_samples:]

In [19]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb


# Train an XGBoost model
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=1000, random_state=42)
model.fit(X_train, y_train)

# Make predictions using the trained model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r_squared = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r_squared}")



Mean Squared Error: 0.1360096599984799
R-squared: 0.7687990141288713


In [27]:
mcrmse_loss(y_test, y_pred, N=3)

0.3677356265103307

In [20]:
wanted = test.drop(['id', 'id_seqpos'], axis=1)


In [21]:
y_pred_test = model.predict(wanted)


In [22]:
y_pred_test

array([[0.5331145 , 1.0575871 , 0.5496925 ],
       [0.6573302 , 0.59932595, 0.71415   ],
       [0.25832835, 0.18345505, 0.17749621],
       ...,
       [0.63137734, 0.593859  , 0.51316845],
       [0.0458054 , 0.1972086 , 0.23234671],
       [0.02224171, 0.24071743, 0.24203265]], dtype=float32)

In [23]:
submission_df = pd.read_csv('/content/sample_submission.csv')
submission_df

,id_seqpos,reactivity,deg_Mg_pH10,deg_Mg_50C
0,id_001f94081_0,0.0,0.0,0.0
1,id_001f94081_4,0.0,0.0,0.0
2,id_001f94081_8,0.0,0.0,0.0
3,id_001f94081_9,0.0,0.0,0.0
4,id_001f94081_13,0.0,0.0,0.0
...,...,...,...,...
32635,id_fff546103_39,0.0,0.0,0.0
32636,id_fff546103_52,0.0,0.0,0.0
32637,id_fff546103_62,0.0,0.0,0.0
32638,id_fff546103_65,0.0,0.0,0.0


In [24]:
submission_df = pd.DataFrame({
    'id_seqpos': test['id_seqpos'],  # Assuming 'test' is your testing data
    'reactivity': y_pred_test[:, 0],  # Predicted values for the 'reactivity' target
    'deg_Mg_pH10': y_pred_test[:, 1],  # Predicted values for the 'deg_Mg_pH10' target
    'deg_Mg_50C': y_pred_test[:, 2]  # Predicted values for the 'deg_Mg_50C' target
})

In [25]:
submission_df.to_csv('submission_change_the_split_way.csv', index=False)